In [2]:
def calcular_frecuencias(texto):
    frecuencias = {}
    for caracter in texto:
        if caracter in frecuencias:
            frecuencias[caracter] += 1
        else:
            frecuencias[caracter] = 1
    return frecuencias


def mostrar_tabla_frecuencias(frecuencias):
    print("TABLA DE FRECUENCIAS")
    print(f"{'Carácter':<15} {'Frecuencia':<15}")
    
    total = sum(frecuencias.values())
    for caracter, freq in sorted(frecuencias.items(), key=lambda x: x[1], reverse=True):
        if caracter == ' ':
            print(f"{'[ESPACIO]':<15} {freq:<15}")
        elif caracter == '\n':
            print(f"{'[SALTO]':<15} {freq:<15}")
        else:
            print(f"{caracter:<15} {freq:<15}")
    print("-"*30)


def crear_nodo(caracter, frecuencia, izquierdo=None, derecho=None):
    #crea un nodo como un diccionario
    return {
        'caracter': caracter,
        'frecuencia': frecuencia,
        'izquierdo': izquierdo,
        'derecho': derecho
    }


def construir_arbol_huffman(frecuencias):
    # Crear lista de nodos iniciales
    nodos = []
    for caracter, frecuencia in frecuencias.items():
        nodos.append(crear_nodo(caracter, frecuencia))
    
    print("CONSTRUCCIÓN DEL ÁRBOL DE HUFFMAN")

    iteracion = 1
    print(f"\nEstado inicial: {len(nodos)} nodos")
    mostrar_nodos(nodos)
    
    # Construir el árbol combinando los dos nodos de menor frecuencia
    while len(nodos) > 1:
        nodos.sort(key=lambda x: x['frecuencia'])
        
        izquierdo = nodos.pop(0)
        derecho = nodos.pop(0)
        
        # nuevo nodo combinado
        nuevo_nodo = crear_nodo(
            None, 
            izquierdo['frecuencia'] + derecho['frecuencia'],
            izquierdo,
            derecho
        )
        
        nodos.append(nuevo_nodo)
        
        print(f"\n--- Iteración {iteracion} ---")
        print(f"Combinando: {obtener_nombre_nodo(izquierdo)} (freq: {izquierdo['frecuencia']}) + "
              f"{obtener_nombre_nodo(derecho)} (freq: {derecho['frecuencia']})")
        print(f"Nuevo nodo con frecuencia: {nuevo_nodo['frecuencia']}")
        print(f"Nodos restantes: {len(nodos)}")
        mostrar_nodos(nodos)
        
        iteracion += 1
    
    print("-"*30)
    return nodos[0]


def obtener_nombre_nodo(nodo):
    if nodo['caracter'] is not None:
        if nodo['caracter'] == ' ':
            return '[ESPACIO]'
        elif nodo['caracter'] == '\n':
            return '[SALTO]'
        else:
            return f"'{nodo['caracter']}'"
    else:
        return '[NODO INTERNO]'


def mostrar_nodos(nodos):
    for nodo in nodos:
        nombre = obtener_nombre_nodo(nodo)
        print(f"  {nombre}: frecuencia = {nodo['frecuencia']}")


def generar_codigos(nodo, codigo_actual='', codigos=None):
    if codigos is None:
        codigos = {}
    
    # Si es un nodo hoja (tiene carácter), guardar el código
    if nodo['caracter'] is not None:
        codigos[nodo['caracter']] = codigo_actual if codigo_actual else '0'
        return codigos
    
    if nodo['izquierdo'] is not None:
        generar_codigos(nodo['izquierdo'], codigo_actual + '0', codigos)
    
    if nodo['derecho'] is not None:
        generar_codigos(nodo['derecho'], codigo_actual + '1', codigos)
    
    return codigos


def mostrar_codigos(codigos):
    print("CÓDIGOS DE HUFFMAN (Libres de Prefijo)")
    print(f"{'Carácter':<15} {'Código':<15} {'Longitud'}")
    print("-"*30)
    
    for caracter, codigo in sorted(codigos.items(), key=lambda x: len(x[1])):
        if caracter == ' ':
            print(f"{'[ESPACIO]':<15} {codigo:<15} {len(codigo)}")
        elif caracter == '\n':
            print(f"{'[SALTO]':<15} {codigo:<15} {len(codigo)}")
        else:
            print(f"{caracter:<15} {codigo:<15} {len(codigo)}")
    print("="*50)


def codificar_texto(texto, codigos):
    texto_codificado = ''
    for caracter in texto:
        texto_codificado += codigos[caracter]
    return texto_codificado


def calcular_tasa_compresion(texto_original, texto_codificado):
    bits_originales = len(texto_original) * 8  
    bits_codificados = len(texto_codificado)
    tasa_compresion = ((bits_originales - bits_codificados) / bits_originales) * 100
    
    print("ANÁLISIS DE COMPRESIÓN")
    print(f"Caracteres en el texto: {len(texto_original)}")
    print(f"Bits originales (8 bits/carácter): {bits_originales} bits")
    print(f"Bits después de Huffman: {bits_codificados} bits")
    print(f"Bits ahorrados: {bits_originales - bits_codificados} bits")
    print(f"Tasa de compresión: {tasa_compresion:.2f}%")
    print(f"Tamaño relativo: {(bits_codificados/bits_originales)*100:.2f}%")
    print("-"*30)


def comprimir_archivo(nombre_archivo):
    try:
        with open(nombre_archivo, 'r', encoding='utf-8') as archivo:
            texto = archivo.read()
        
        if not texto:
            print("El archivo está vacío.")
            return
        
        print(f"\nArchivo '{nombre_archivo}' cargado exitosamente.")
        print(f"Tamaño del texto: {len(texto)} caracteres")
        
        comprimir_texto(texto)
        
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{nombre_archivo}'")
    except Exception as e:
        print(f"Error al leer el archivo: {e}")


def comprimir_texto(texto):
    frecuencias = calcular_frecuencias(texto)
    mostrar_tabla_frecuencias(frecuencias)
    
    arbol = construir_arbol_huffman(frecuencias)
    
    codigos = generar_codigos(arbol)
    mostrar_codigos(codigos)
    
    texto_codificado = codificar_texto(texto, codigos)
    
    print("TEXTO CODIFICADO")
    
    # Mostrar solo los primeros 500 caracteres si es muy largo
    if len(texto_codificado) > 500:
        print(texto_codificado[:500] + "...")
        print(f"\n(Mostrando solo los primeros 500 bits de {len(texto_codificado)} totales)")
    else:
        print(texto_codificado)
    print("="*50)
    
    calcular_tasa_compresion(texto, texto_codificado)
    
    return texto_codificado, codigos

def main():
    print("PROGRAMA DE CODIFICACIÓN HUFFMAN")

    print("\n¿Qué deseas hacer?")
    print("1. Comprimir texto introducido manualmente")
    print("2. Comprimir archivo de texto")
    
    opcion = input("\nElige una opción: ").strip()
    
    if opcion == '1':
        texto = input("Introduce el texto a comprimir: ")
        
        if not texto:
            print("No se introdujo ningún texto.")
        else:
            comprimir_texto(texto)
    
    elif opcion == '2':
        nombre_archivo = input("Introduce el nombre del archivo (ej: texto.txt): ").strip()
        comprimir_archivo(nombre_archivo)
    
    else:
        print("Opción no válida. El programa finalizará.")

if __name__ == "__main__":
    main()

PROGRAMA DE CODIFICACIÓN HUFFMAN

¿Qué deseas hacer?
1. Comprimir texto introducido manualmente
2. Comprimir archivo de texto
TABLA DE FRECUENCIAS
Carácter        Frecuencia     
g               8              
[ESPACIO]       7              
d               6              
f               6              
m               5              
i               4              
k               4              
s               3              
h               3              
a               3              
t               3              
o               3              
n               2              
e               2              
r               2              
u               2              
z               2              
w               1              
v               1              
q               1              
b               1              
l               1              
j               1              
,               1              
------------------------------
CONSTRUCCIÓN DEL ÁRBOL